Hierarchical evaluation
========

In [ ]:
#use full width of screen in Jupyter notebooks
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

import numpy as np
import pandas as pd
import glob
import os.path
import sys
import math

%matplotlib inline
from plotting.plots import *

from svecon.HierarchicalGridSearchCV import HierarchicalGridSearchCV
from svecon.EmptyTransformer import EmptyTransformer

from sklearn.cross_validation import StratifiedKFold
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import Imputer, StandardScaler, LabelEncoder

from metric_learn import LMNN, NCA, LFDA, Covariance#, CMAES, FullMatrixTransformer, NeuralNetworkTransformer
from metric_learn import ITML_Supervised, SDML_Supervised, LSML_Supervised, RCA_Supervised

import plotly
plotly.tools.set_credentials_file(username='sveco', api_key='8701ghzf0i')
import plotly.offline as py
import plotly.graph_objs as go
py.init_notebook_mode()

datasetsDirectory = 'datasets'
resultsDirectory = 'datasets-results-all'
dumpsDirectory = 'datasets-results-dumps'

if not os.path.exists(resultsDirectory):
    os.makedirs(resultsDirectory)

if not os.path.exists(dumpsDirectory):
    os.makedirs(dumpsDirectory)

default_n_jobs = 8
default_random_state = 789
default_n_folds = 10
default_shuffle = True

import logging
logger = logging.getLogger()
logger.setLevel(logging.DEBUG)
logger.handlers = []
fh = logging.FileHandler("{}/error.log".format(resultsDirectory))
fh.setLevel(logging.DEBUG)
logger.addHandler(fh)
fh = logging.StreamHandler(sys.stdout)
fh.setLevel(logging.ERROR)
logger.addHandler(fh)

import pickle
def save_object(obj, filename):
    with open("{}/dump_{}.bin".format(dumpsDirectory,filename), 'wb') as output:
        logger.info("saving file" + filename)
        pickle.dump(obj, output, pickle.HIGHEST_PROTOCOL)

Statistics
===========

In [ ]:
resultsMask = "{}/*_result.csv".format(resultsDirectory)
alldatatable = pd.concat([pd.DataFrame.from_csv(x) for x in sorted(glob.glob(resultsMask))])

# BUGFIX - ZERO SCORES
# scores = [ x for x in alldatatable.columns.values if x[:5]=='score']
# alldatatable['mean'] = alldatatable[scores].mean(axis=1)

alldata = alldatatable.set_index(['dataset','technique', alldatatable.index])
alldata

In [ ]:
sortedTechniques = [
    ('kNN', 'Euclidean'),
    ('Cov+kNN', 'Covariance'),
    ('LMNN+kNN', 'LMNN'),
    ('CMAES+kNN', 'CMA-ES'),
    ('NCA+kNN', 'NCA'),
    ('LFDA+kNN', 'LFDA'),
    ('ITML+kNN', 'ITML'),
    ('LSML+kNN', 'LSML'),
    ('RCA+kNN', 'RCA'),
    ('SDML+kNN', 'SDML'),
    ('stand+kNN', 'norm Euclidean'),
    ('stand+Cov+kNN', 'norm Covariance'),
    ('stand+LMNN+kNN', 'norm LMNN'),
    ('stand+CMAES+kNN', 'norm CMA-ES'),
    ('stand+NCA+kNN', 'norm NCA'),
    ('stand+LFDA+kNN', 'norm LFDA'),
    ('stand+ITML+kNN', 'norm ITML'),
    ('stand+LSML+kNN', 'norm LSML'),
    ('stand+RCA+kNN', 'norm RCA'),
    ('stand+SDML+kNN', 'norm SDML'),
]

sortedTechniques = [
    ('kNN', 'Euclidean'),
    ('stand+kNN', 'norm Euclidean'),
    
    ('Cov+kNN', 'Covariance'),
    ('stand+Cov+kNN', 'norm Covariance'),
    
    ('LMNN+kNN', 'LMNN'),
    ('stand+LMNN+kNN', 'norm LMNN'),
    
    ('CMAES+kNN', 'CMA-ES'),
    ('stand+CMAES+kNN', 'norm CMA-ES'),
    
    ('NCA+kNN', 'NCA'),
    ('stand+NCA+kNN', 'norm NCA'),
    
    ('LFDA+kNN', 'LFDA'),
    ('stand+LFDA+kNN', 'norm LFDA'),
    
    ('ITML+kNN', 'ITML'),
    ('stand+ITML+kNN', 'norm ITML'),
    
    ('LSML+kNN', 'LSML'),
    ('stand+LSML+kNN', 'norm LSML'),
    
    ('RCA+kNN', 'RCA'),
    ('stand+RCA+kNN', 'norm RCA'),
    
    ('SDML+kNN', 'SDML'),
    ('stand+SDML+kNN', 'norm SDML'),
]

# Best run successrate per dataset

In [ ]:
showBestParams = False
showNorm = True
for dataset in alldata.index.levels[0]:
    traces = []
    means = []
    techniques = []
    boxdata = []
    skipped = []
    for i, (techniqueIndex, technique) in enumerate(sortedTechniques):
        techniqueRows = alldata.loc[dataset].loc[techniqueIndex]
        if techniqueRows.shape[0]==0:
            skipped.append(i)
            continue
#         if technique[:4] == 'norm':
#             skipped.append(i)
#             continue
        
        bestRowNum = techniqueRows['mean'].argmax()
        bestRow = techniqueRows.iloc[bestRowNum,:]
        
        scores = [ x for x in alldata.columns.values if x[:5]=='score']
        train_scores = [ x for x in alldata.columns.values if x[:11]=='train_score']
        if sum(bestRow[scores].values)==0:
            skipped.append(i)
            continue
            
        means.append( bestRow['mean'] ) # if bestRow['mean']>0 else np.median(bestRow[scores].values)
#         means.append( bestRow['train_mean'] )
        techniques.append( technique.replace('+kNN', '').replace('kNN', 'Euclid') )
#         techniques.append( technique+' [train]' )
        boxdata.append([x for x in bestRow[scores].values if x > 0])
        
#         if technique == 'CMA-ES':
#             cmaes = [x for x in bestRow[scores].values if x > 0]
#         if technique == 'Euclidean':
#             euc = [x for x in bestRow[scores].values if x > 0]
        
        if showBestParams:
            print("======",dataset,"======",technique,"======")
            print(bestRow)
            print()

#     import scipy.stats
#     z, p = scipy.stats.mannwhitneyu(cmaes, euc)
#     p_value = p * 2
#     s = stars(p)
#     print(z,p,s)
            
    plotBox(dataset, boxdata, techniques, means, doubleColors=showNorm, skipped=skipped)
    break

# Average success rate across all datasets (from best runs only!)

In [ ]:
valuesPerTechnique = {}
for dataset in alldata.index.levels[0]:
    for technique, techniqueName in sortedTechniques:
        techniqueRows = alldata.loc[dataset].loc[technique]
        if techniqueRows.shape[0]==0:
            continue
            
        bestRowNum = techniqueRows['mean'].argmax()
        bestRow = techniqueRows.iloc[bestRowNum,:]
        scores = [ x for x in alldata.columns.values if x[:5]=='score']
        
        if technique not in valuesPerTechnique:
            valuesPerTechnique[technique] = {'means':[], 'scores':[]}
        
        # SKIP ALL WHERE THERE IS SUCCESSRATE 0 IN ANY FOLD
        if any(x == 0 for x in bestRow[scores].values):
            continue
        
        valuesPerTechnique[technique]['means'].append(bestRow['mean'])
        valuesPerTechnique[technique]['scores'].extend(bestRow[scores].values)

# traces = []
# means = []
# for technique,value in sortedTechniques:
#     traces.append( go.Box(
#             y=valuesPerTechnique[technique]['scores'],
#             name=technique,
#     ))
#     means.append((technique, np.mean(valuesPerTechnique[technique]['means'])))
    
# traces.append( go.Scatter( 
#         x=[x[0] for x in means],
#         y=[x[1] for x in means],
#         mode='lines', name='mean', marker=dict(color='black')
#     ))
    
# layout = go.Layout(
#     yaxis=dict(title='successrate', zeroline=False ),
#     title='Aggregated',
# )

# fig = go.Figure(data=traces, layout=layout)
# py.iplot(fig)
# # plotly.plotly.image.save_as(fig, filename='{}/{}-plot.png'.format(resultsDirectory, dataset))

boxdata = []
means = []
filteredTechniques = []
for technique,value in sortedTechniques:
    boxdata.append(valuesPerTechnique[technique]['scores'])
    means.append(np.mean(valuesPerTechnique[technique]['means']))
    filteredTechniques.append(value)

plotBox('Aggregated scores', boxdata, filteredTechniques, means, doubleColors=showNorm, skipped=skipped)

# Times for best run per dataset

In [ ]:
for dataset in alldata.index.levels[0]:
    
    traces = []
    for timeCol in [ x for x in alldata.columns.values if x[:4]=='time']:
        times = []
        for technique in alldata.index.levels[1]:
            techniqueRows = alldata.loc[dataset].loc[technique]
            if techniqueRows.shape[0]==0:
                continue
                
            bestRowNum = techniqueRows['mean'].argmax()
            bestRow = techniqueRows.iloc[bestRowNum,:]
            
            times.append( bestRow[timeCol] )

        times = [0 if np.isnan(x) else x for x in times]
            
        traces.append(
            go.Bar(
                y=times,
                x=alldata.index.levels[1],
                name=timeCol
            )
        )
        
    layout = go.Layout(
        barmode='stack',
        yaxis=dict(
            title='seconds',
            zeroline=False
        ),
        title=dataset,
    )
    fig = go.Figure(data=traces, layout=layout)
    py.iplot(fig)
#     plotly.plotly.image.save_as(fig, filename='{}/{}-plot.png'.format(resultsDirectory, dataset))


# Average times per technique per dataset

In [ ]:
for dataset in alldata.index.levels[0]:
    
    traces = []
    for timeCol in [ x for x in alldata.columns.values if x[:4]=='time']:
        times = []
        for technique in alldata.index.levels[1]:
            techniqueRows = alldata.loc[dataset].loc[technique]
            if techniqueRows.shape[0]==0:
                continue
            
            times.append( techniqueRows[timeCol].mean() )

        times = [0 if np.isnan(x) else x for x in times]
            
        traces.append(
            go.Bar(
                y=times,
                x=alldata.index.levels[1],
                name=timeCol
            )
        )
        
    layout = go.Layout(
        barmode='stack',
        yaxis=dict(
            title='seconds',
            zeroline=False
        ),
        title=dataset,
    )
    fig = go.Figure(data=traces, layout=layout)
    py.iplot(fig)
#     plotly.plotly.image.save_as(fig, filename='{}/{}-plot.png'.format(resultsDirectory, dataset))


# Looking at k param (kNN)

In [ ]:
alldataknn = alldatatable.set_index(['dataset','technique', 'knn__n_neighbors', alldatatable.index])
alldataknn

# Success rates for technique per dataset per k param in kNN (a lot of graphs!)

In [ ]:
showBestParams = False
for dataset in alldataknn.index.levels[0]:
    for techniqueIndex, technique in sortedTechniques: 
        traces = []
        means = []
        techniques = []
        boxdata = []
        for k in alldataknn.index.levels[2]:
            techniqueRows = alldataknn.loc[dataset].loc[techniqueIndex].loc[k]            
            
            if techniqueRows.shape[0]==0:
                continue
            bestRowNum = techniqueRows['mean'].argmax()
            bestRow = techniqueRows.loc[bestRowNum,:]

            scores = [ x for x in alldataknn.columns.values if x[:5]=='score']
    #         scores = [ x for x in alldata.columns.values if x[:11]=='train_score']
#             if sum(bestRow[scores].values)==0: continue

            means.append( bestRow['mean'] ) # if bestRow['mean']>0 else np.median(bestRow[scores].values)
    #         means.append( bestRow['train_mean'] )
#             techniques.append( "k={}".format(k) )
            techniques.append(k)
            boxdata.append([x for x in bestRow[scores].values if x > 0])
#             traces.append(
#                 go.Box(
#                     y=[x for x in bestRow[scores].values if x > 0],
#                     name="k={}".format(k),
#                 )
#             )

            if showBestParams:
                print("======",dataset,"======",technique,"======")
                print(bestRow)
                print()

#         traces.append( go.Scatter( x=techniques, y=means, mode='lines', name='mean', marker=dict(color='black') ))
#         layout = go.Layout(
#             yaxis=dict(title=technique+' successrate', zeroline=False ),
#             title=dataset,
#         )
#         fig = go.Figure(data=traces, layout=layout)
#         py.iplot(fig)
# #     plotly.plotly.image.save_as(fig, filename='{}/{}-plot.png'.format(resultsDirectory, dataset))

        plotBox("{}: {}".format(technique, dataset), boxdata, techniques, means, xlabel='"k" in kNN', doubleColors=False, rotateLabels=0)
    
        break
    break

# Successrates for each dataset depending on k param (kNN)

In [ ]:
showBestParams = False
for dataset in alldataknn.index.levels[0]:
    traces = []
    filteredTechniques = []
    data = []
    for techniqueIndex, technique in sortedTechniques: 
        if techniqueIndex[:5]=="stand": continue
        
        means = []
        for k in alldataknn.index.levels[2]:
            techniqueRows = alldataknn.loc[dataset].loc[techniqueIndex].loc[k]            
            if techniqueRows.shape[0]==0: continue
            bestRowNum = techniqueRows['mean'].argmax()
            bestRow = techniqueRows.loc[bestRowNum,:]
    
            means.append( bestRow['mean'] )

#         k_label = ['k='+str(x) for x in alldataknn.index.levels[2]]
        k_label = alldataknn.index.levels[2]
        data.append(means)
        filteredTechniques.append(technique)
#         traces.append( go.Scatter( x=k_label, y=means, mode='lines', name=technique ))

#     layout = go.Layout(
#         yaxis=dict(title='successrate', zeroline=False ),
#         title=dataset,
#     )
#     fig = go.Figure(data=traces, layout=layout)
#     py.iplot(fig)
# #     plotly.plotly.image.save_as(fig, filename='{}/{}-plot.png'.format(resultsDirectory, dataset))
    plotLines(dataset, data, filteredTechniques, k_label, doubleColors=False, xlabel='"k" in kNN')
    break

# Successrate depending on parameter

In [ ]:
interesting_params = [
#     ('lmnn', 'k'),
#     ('lmnn', 'max_iter'),
#     ('lmnn', 'regularization'),
#     ('lmnn', 'max_iter'),
# #     ('lmnn', 'learn_rate'),
#     ('itml', 'num_constraints'),
#     ('itml', 'gamma'),
#     ('itml', 'max_iters'),
#     ('sdml', 'num_constraints'),
#     ('sdml', 'use_cov'),
#     ('sdml', 'balance_param'),
#     ('sdml', 'sparsity_param'),
#     ('lsml', 'num_constraints'),
#     ('lsml', 'max_iter'),
#     ('nca', 'max_iter'),
#     ('nca', 'learn_rate'),
#     ('lfda', 'metric'),
# #     ('rca', 'num_chunks'),
# #     ('rca', 'chunk_size'),
#     ('cmaes', 'metric'),
#     ('cmaes', 'n_gen'),
    ('cmaes', ['n_neighbors', 'c__n_neighbors']),
#     ('cmaes', 'knn_weights'),
]

for technique,paramNames in interesting_params:
    if not isinstance(paramNames, (list, tuple)):
        paramNames = [paramNames]
    
    traces = []
    data = []
    for dataset in alldata.index.levels[0]:
        techniqueFullName = '{}+kNN'.format(technique.upper())

        means = []
        for paramName in paramNames:
            possibleParamValues = alldata['{}__{}'.format(technique, paramName)].dropna().unique()
            for param in possibleParamValues:
                techniqueRows = alldataknn.loc[dataset].loc[techniqueFullName]
                techniqueRows = techniqueRows[techniqueRows['{}__{}'.format(technique, paramName)] == param]
                if techniqueRows.shape[0]==0: continue
                bestRowNum = techniqueRows['mean'].argmax()
                bestRow = techniqueRows.loc[bestRowNum,:]

                means.append( bestRow['mean'] )

        param_labels = ['{}={}'.format(paramNames[0], x) for x in possibleParamValues]
        param_labels = possibleParamValues
#         traces.append( go.Scatter( x=param_labels, y=means, mode='lines', name=dataset ))
        data.append(means)

#     layout = go.Layout(
#         yaxis=dict(title='successrate', zeroline=False ),
#         title='{} by {}'.format(technique.upper(), paramName),
#     )
#     fig = go.Figure(data=traces, layout=layout)
#     py.iplot(fig)
# #     plotly.plotly.image.save_as(fig, filename='{}/{}-plot.png'.format(resultsDirectory, dataset))
    datasetL = alldata.index.levels[0]
    plotLines('{1} in {0}'.format(technique, paramNames[0]), data, datasetL, param_labels, doubleColors=False, xlabel=paramNames[0])

# Learning times for all techniques per dataset

In [ ]:
showBestParams = False
traces = []
for technique in alldata.index.levels[1]: 
    times = []
    datasetNames = []
    for dataset in alldata.index.levels[0]:
    
        if technique in ['kNN', 'stand+kNN']: continue
        if dataset in ['mice-protein']: continue
#         if technique[:5]!="stand": continue
        
        techniqueRows = alldataknn.loc[dataset].loc[technique]
        if techniqueRows.shape[0]==0: continue

        bestRowNum = techniqueRows['mean'].argmax()
        bestRow = techniqueRows.loc[bestRowNum,:]

        times.append( bestRow['time2']/3600 )
        datasetNames.append(dataset)
    
    traces.append( go.Scatter( x=datasetNames, y=times, mode='lines', name=technique ))

layout = go.Layout(
    yaxis=dict(title='hours', zeroline=False ),
    title='Learning times',
)
fig = go.Figure(data=traces, layout=layout)
py.iplot(fig)
    
# #     plotly.plotly.image.save_as(fig, filename='{}/{}-plot.png'.format(resultsDirectory, dataset))
#     break

# Training time depending on parameter

In [ ]:
interesting_params = [
    ('lmnn', 'k'),
#     ('lmnn', 'max_iter'),
#     ('lmnn', 'regularization'),
#     ('lmnn', 'max_iter'),
# #     ('lmnn', 'learn_rate'),
#     ('itml', 'num_constraints'),
#     ('itml', 'gamma'),
#     ('itml', 'max_iters'),
#     ('sdml', 'num_constraints'),
#     ('sdml', 'use_cov'),
#     ('sdml', 'balance_param'),
#     ('sdml', 'sparsity_param'),
#     ('lsml', 'num_constraints'),
#     ('lsml', 'max_iter'),
# #     ('nca', 'max_iter'),
# #     ('nca', 'learn_rate'),
#     ('lfda', 'metric'),
# #     ('rca', 'num_chunks'),
# #     ('rca', 'chunk_size'),
#     ('cmaes', 'metric'),
#     ('cmaes', 'n_gen'),
#     ('cmaes', 'n_neighbors'),
#     ('cmaes', 'knn_weights'),
]

for technique,paramName in interesting_params:
    traces = []
    for dataset in alldata.index.levels[0]:
        techniqueFullName = '{}+kNN'.format(technique.upper())

        means = []
        possibleParamValues = alldata['{}__{}'.format(technique, paramName)].dropna().unique()
        for param in possibleParamValues:
            techniqueRows = alldataknn.loc[dataset].loc[techniqueFullName]
            techniqueRows = techniqueRows[techniqueRows['{}__{}'.format(technique, paramName)]==param]
            if techniqueRows.shape[0]==0: continue

            means.append( techniqueRows['time2'].mean() /3600 )

        param_labels = ['{}={}'.format(paramName, x) for x in possibleParamValues]
        traces.append( go.Scatter( x=param_labels, y=means, mode='lines', name=dataset ))

    layout = go.Layout(
        yaxis=dict(title='hours', zeroline=False ),
        title='{} by {}'.format(technique.upper(), paramName),
    )
    fig = go.Figure(data=traces, layout=layout)
    py.iplot(fig)
#     plotly.plotly.image.save_as(fig, filename='{}/{}-plot.png'.format(resultsDirectory, dataset))